In [1]:
import core.metrics as metrics
from models import * 

metric = getattr(metrics, 'mAP')
print(metric)

<class 'core.metrics.oksap.OKSAP'>


/home/otter/.local/share/mamba/envs/hpe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch 
N, K = 100, 17
gt = torch.rand(N, K, 3) 
gt[..., 2] = torch.randint(0, 3, (N, K))/10      # visibility 0~2
pred = gt[..., :2] + torch.randn(N, K, 2) * 0.9 # noisy prediction
areas = torch.rand(N) * 5
scores = torch.rand(N)

sigmas = torch.ones(K) / K
ap_metric = metric(sigmas, oks_thresholds=[0.5])   
mean_ap = ap_metric.compute(gt, pred, areas, scores)

print("mAP50:", mean_ap)

ap_metric = metric(sigmas)                      # 기본: OKS 0.50~0.95
mean_ap = ap_metric.compute(gt, pred, areas, scores)

print("mAP50-95:", mean_ap)

50.0